# 02. Introduction to Data Analysis with Scala and Spark

- 데이터 클린징은 데이터 사이언스 프로젝트에서 첫번째 단계이며, 매우 중요한 단계임.
- 현명한 분석가는 기초 데이터품질에 문제가 있거나,  편의 또는 리얼 데이터가 아니라고 판단되어지만 데이터분석을 취소함.


- 이렇게 중요함에도 많은 교재와 강의에서 데이터 클린징은 언급되지 않거나 데이터 주어진 것으로 넘어감.
- 데이터 클린징은 정말 지루하고 지루하고 지루한 작업.
- 많은 새로운 데이터 과학자들은 최소한의 허용된 상태의 데이터만을 가지고 분석을 바로 시작해서 엉뚱한 결과가 얻은 후에 데이터 품질에 심각한 문제가 있다라고 인식함.


- garbage in, garbage out


- 데이터 과학자은 재미있고 가치있는 문제을 발견할 수 있는 능력이 필요


- ML 알고리즘, 실시간분석, Web-Scale 그래프분석을 할 수 있는 Spark가 데이터 클린징과 가치있는 문제를 발견할 수 있는데 도움이 될것임.

## Scala for Data Scientists
- Spark 는  R 또는 Python에서 사용이 가능하고, PySpark도 있음.
- 그러나, Spark 프레임워크와 동일한 언어로 Spark을 학습하는것이 많은 이점이 있어서 중요한 예제들은 Scala로 작성됨.
     - 성능의 오버헤드를 감소
     - 최신버전에서 가장 좋은 접근방법을 제공
     - Spark 철학을 이해하는데 도움.

## The Spark Programming Model
- Spark program을 작성하는 일반적인 요소
    - 1) 입력데이터 셋에서  필요한 형식으로 데이터 변환을 정의
    - 2) 변환된 데이터를 저장소 또는 메모리로 결과를 출력하는 action을 포함.
    - 3) 분산방식으로 얻어진 결과를  운영하기 위한  로컬 컴퓨팅.
    
- Spark는  저장소와 실행에 대한 2가지 추상화 프레임워크 사이를 상호동작을 제공함.

## Record Linkage

## Downloading and installing Spark 1.5.2

### Java Development Kit 7 (JDK) 설치
- http://www.oracle.com/technetwork/java/javase/downloads/jdk7-downloads-1880260.html 
- sudo apt-get install openjdk-7-jdk

### download the latest release of Spark
- https://spark.apache.org/downloads.html
- Pre-built for Hadoop 2.6 and later과 Direct Download을 선택
- wget http://apache.mirror.cdnetworks.com/spark/spark-1.5.2/spark-1.5.2-bin-hadoop2.6.tgz
- tar xvf spark-1.5.2-bin-hadoop2.6.tgz
- ln -s spark-1.5.2-bin-hadoop2.6 spark
- cd spark 
- ls 

### Spark의 내부 디렉토리 구조
- core : sparkAPI와 핵심 구성요소의 소스코드가 포함됨.
- bin : spark app과 spark shell을 실행시키기 위한 실행파일들이 포함됨.
- graphx, mllib, sql, streaming : graph processing, machine learning, queries, and stream processing에 관련된 LIbrary
- example : spark app의 예제와 데모를 포함.
- conf : slave 노드와 기타 설정파일이 포함.

### Spark 경로 설정
- ~/.bashrc  or ~/.bash_profile
```
export SPARK_HOME="/home/deepbio/app/spark/"
```

- source ~/.bashrc or ~/.bash_profile

## Getting Started: The Spark Shell and SparkContext

- UC Irvine Machine Learning Repository에서부터 얻은 샘플데이터를 사용함.
- 이 데이터는 2010년에 German hospital에서 수행한 record linkage 연구에서 설계되어짐.
- 환자 이름, 주소, 생년월일 같은 정보로 환자 의료 기록들을 매칭함.
- 매칭한 필드는 얼마큼 비슷한지를 0.0 ~ 1.0 사이의 숫자값으로 할당됨.  수작업으로 이루어짐.

In [1]:
$ mkdir linkage
$ cd linkage/
$ curl -o donation.zip http://bit.ly/1Aoywaq
$ unzip donation.zip
$ unzip 'block_*.zip'


$ hadoop fs -mkdir linkage
$ hadoop fs -put block_*.csv linkage

In [11]:
! ls -al ~/note/work/linkage/

total 256400
drwxrwxr-x 2 deepbio deepbio     4096 Dec 16 18:22 .
drwxr-xr-x 6 deepbio deepbio     4096 Dec 16 18:22 ..
-rw-r--r-- 1 deepbio deepbio 26255957 Mar  9  2011 block_10.csv
-rw-r--r-- 1 deepbio deepbio 26248574 Mar  9  2011 block_1.csv
-rw-r--r-- 1 deepbio deepbio 26241784 Mar  9  2011 block_2.csv
-rw-r--r-- 1 deepbio deepbio 26253247 Mar  9  2011 block_3.csv
-rw-r--r-- 1 deepbio deepbio 26247471 Mar  9  2011 block_4.csv
-rw-r--r-- 1 deepbio deepbio 26249424 Mar  9  2011 block_5.csv
-rw-r--r-- 1 deepbio deepbio 26256126 Mar  9  2011 block_6.csv
-rw-r--r-- 1 deepbio deepbio 26261911 Mar  9  2011 block_7.csv
-rw-r--r-- 1 deepbio deepbio 26253911 Mar  9  2011 block_8.csv
-rw-r--r-- 1 deepbio deepbio 26254012 Mar  9  2011 block_9.csv


In [6]:
! head ~/note/work/linkage/block_1.csv

"id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match"
37291,53113,0.833333333333333,?,1,?,1,1,1,1,0,TRUE
39086,47614,1,?,1,?,1,1,1,1,1,TRUE
70031,70237,1,?,1,?,1,1,1,1,1,TRUE
84795,97439,1,?,1,?,1,1,1,1,1,TRUE
36950,42116,1,?,1,1,1,1,1,1,1,TRUE
42413,48491,1,?,1,?,1,1,1,1,1,TRUE
25965,64753,1,?,1,?,1,1,1,1,1,TRUE
49451,90407,1,?,1,?,1,1,1,1,0,TRUE
39932,40902,1,?,1,?,1,1,1,1,1,TRUE


In [ ]:
# Hadoop 2.6 + spark에서 할때는 
bin/hdfs dfsadmin -report
bin/yarn node -list

bin/hdfs dfs  -mkdir /linkage
bin/hdfs dfs -put ~/linkage/*  /linkage
bin/hdfs dfs -ls  /linkage
bin/hdfs dfs  -tail  /linkage/block_1.csv

- spark 설치 및 실행은 다른 문서로 대치함.
- linkage 데이터를 읽어오기

In [ ]:
# val 과 var 의 비교 설명
val rawblocks = sc.textFile("linkage")
rawblocks = sc.textFile("linkage")

var varblocks = sc.textFile("linkage")
varblocks = sc.textFile("linkage")

val rawblocks = sc.textFile("linakge")
val rawblocks = sc.textFile("linkage")

## Bringing Data from the Cluster to the Client

In [ ]:
# Spark shell 시작
bin/spark-shell  --driver-memory 2g

# 하둡
val rawblocks = sc.textFile("hdfs://node01:9000/linkage")

# local mode
val rawblocks = sc.textFile("linkage")
rawblocks.first

# 전체데이터에서 10개만 데이터를 로컬로 가지고 옴.
val head = rawblocks.take(10)
head.foreach(println)

 - scala에서는 타임을 compiler가 예상할 수 있으면 생략 가능,  축약된 함수정의와 잘 정의된 함수정의를 확인. 

In [ ]:
def isHeader(line: String) = line.contains("id_1")

def isHeader(line: String): Boolean = {
    line.contains("id_1")
}

- 헤더정보를 출력해보고, 헤더가 아닌 정보도 출력하는 2가지 방법을 알아보자.
- scala의 람다표현식으로 filter는 하나의 인자를 받아서 true와 false을 리턴하는 함수를 인자로 받음.
- isHeader가 하나의 인자만을 필요하기 때문에 타임을 예상할 수 있어서 생략가능함.

In [ ]:
head.filter(isHeader).foreach(println)
head.filter(x => isHeader(x)).length

head.filterNot(isHeader).length
head.filter(x => !isHeader(x)).length
head.filter(!isHeader(_)).length

## Shipping Code from the Client to the Cluster

- spark에서는 cluster에서 동작하는 코드나 local machine에서 동작하는 코드나 정확히 일치함.
- 아주 강력함을 제공
    - 대화형식으로 개발이 가능함.
    - 샘플데이터 셋으로 개발한 후에 실제 데이터로 적용이 바로 가능함

In [ ]:
# 실제 데이터에서 동작시킴.
val noheader = rawblocks.filter(x => !isHeader(x))

noheader.first

## Structuring Data with Tuples and Case Classes
- scala에서는 python과 같이 튜플 타임을 제공함.
- 또한, case classes을 제공해서 record 정보를 편리하게 사용할 수 있게 함.
     - immutable class 한 단순한 클래스
     - 자바와 같이 toString, equals, hashCode와 같은 함수를 제공

In [ ]:
# 데이터를 가공하는 기본적인 방법들을 소개함.
val line = head(5)
val pieces = line.split(',')
val id1 = pieces(0).toInt
val id2 = pieces(1).toInt
val matched = pieces(11).toBoolean
val rawscores = pieces.slice(2, 11)

# 에러 발생함.
rawscores.map(s => s.toDouble)

#에러에 대한 처리함.
def toDouble(s: String) = {
   if ("?".equals(s)) Double.NaN else s.toDouble
}
val scores = rawscores.map(toDouble)

In [ ]:
# 튜플 형식으로 데이터를 가공

def parse(line: String) = {
  val pieces = line.split(',')
  val id1 = pieces(0).toInt
  val id2 = pieces(1).toInt
  val scores = pieces.slice(2, 11).map(toDouble)
  val matched = pieces(11).toBoolean
  (id1, id2, scores, matched)
}
val tup = parse(line)
tup._1
tup.productElement(0)
tup.productArity

In [ ]:
# case class 형식으로 데이터를 가공
case class MatchData(id1: Int, id2: Int, scores: Array[Double], matched: Boolean)

def parse(line: String) = {
  val pieces = line.split(',')
  val id1 = pieces(0).toInt
  val id2 = pieces(1).toInt
  val scores = pieces.slice(2, 11).map(toDouble)
  val matched = pieces(11).toBoolean
  MatchData(id1, id2, scores, matched)
}
val md = parse(line)
md.matched
md.id1

In [ ]:
# client내부의 데이터로 확인
val mds = head.filter(x => !isHeader(x)).map(x => parse(x))
mds(0)

# 실제 데이터로 cluster에서 동작시킴.
val parsed = noheader.map(line => parse(line))
parsed.first

In [ ]:
# RDD 데이터를 메모리에 캐싱할 수 있도록 cache함수를 제공함.
# 자주 사용하는 데이터는 cache하는 것이 좋음.
parsed.cache()

## Aggregations

- 데이터를 그룹핑하고 그룹별 합산연산을 수행
- groupBy()함수를 통해서  Scala Map[Boolean, Array[MatchData]]에 true인 경우와 false 인 경우가 나누어짐.
- mapValues()함수에서  true경우의 사이즈와 false 경우의 사이즈값을 출력함.

In [ ]:
val grouped = mds.groupBy(md => md.matched)
grouped.mapValues(x => x.size).foreach(println)

val groupedRDD = parsed.groupBy(md => md.matched)
groupedRDD.mapValues(x => x.size).foreach(println)

- 클러스터상에서 aggregation을 수행할때는 여러개의 머신에서 수행된다는 것을 기억해야 함.
- aggregations 작업은 각각 머신에서 네트워크를 통해서 데이터 이동이 필수임.
- 데이터 이동은 데이터의 serializing, compressing, sending, decompressing, deserializing와 같은 많은 작업을 수행함.
- 이와 같은 작업을 빠르게 처리하기 위해서는 데이터의 이동을 최소화함.
- aggregations전에 필요한 데이터만을 filtering을 처리함.

## Creating Histograms
- MatchData 레코드들이 field 필드에 대해서 true와 falsed의 개수가 얼마큼인지 세어보자.
- RDD[T] 에서는 countByValue()함수에서 Map[T,Long]  형식으로 리턴함.

- Map[T,Long]은  sort 함수를 제공하지 않아서 Scala의 Seq type으로 변환함.
- Scala은 Seq은 자바의 List 인터페이스와 비슷함.

In [ ]:
val matchCounts = parsed.map(md => md.matched).countByValue()
val matchCountsSeq = matchCounts.toSeq

matchCountsSeq.sortBy(_._1).foreach(println)
matchCountsSeq.sortBy(_._2).foreach(println)
matchCountsSeq.sortBy(_._2).reverse.foreach(println)

In [ ]:
# 참고 : Word Count 예제로  map()함수의 기능을 알아봄.
val textFile = spark.textFile("hdfs://...")
val counts = textFile.flatMap(line => line.split(" "))
                 .map(word => (word, 1))
                 .reduceByKey(_ + _)
counts.saveAsTextFile("hdfs://...")

In [ ]:
# 참고 : hadoop MapReduce로  WordCount 구현
public class WordCount {

  public static class TokenizerMapper
       extends Mapper<Object, Text, Text, IntWritable>{

    private final static IntWritable one = new IntWritable(1);
    private Text word = new Text();

    public void map(Object key, Text value, Context context
                    ) throws IOException, InterruptedException {
      StringTokenizer itr = new StringTokenizer(value.toString());
      while (itr.hasMoreTokens()) {
        word.set(itr.nextToken());
        context.write(word, one);
      }
    }
  }

  public static class IntSumReducer
       extends Reducer<Text,IntWritable,Text,IntWritable> {
    private IntWritable result = new IntWritable();

    public void reduce(Text key, Iterable<IntWritable> values,
                       Context context
                       ) throws IOException, InterruptedException {
      int sum = 0;
      for (IntWritable val : values) {
        sum += val.get();
      }
      result.set(sum);
      context.write(key, result);
    }
  }

  public static void main(String[] args) throws Exception {
    Configuration conf = new Configuration();
    Job job = Job.getInstance(conf, "word count");
    job.setJarByClass(WordCount.class);
    job.setMapperClass(TokenizerMapper.class);
    job.setCombinerClass(IntSumReducer.class);
    job.setReducerClass(IntSumReducer.class);
    job.setOutputKeyClass(Text.class);
    job.setOutputValueClass(IntWritable.class);
    FileInputFormat.addInputPath(job, new Path(args[0]));
    FileOutputFormat.setOutputPath(job, new Path(args[1]));
    System.exit(job.waitForCompletion(true) ? 0 : 1);
  }
}


## Summary Statistics for Continuous Variables

- Patient records에 있는 match score와 연속형 변수에 대해서,  평균, 표준편차, 최대값, 최소값 같은 분표에 대한 통계치를 사용함.
- RDD[Double]에서는 state()함수를 지원하여 요약통계량을 구해줌.

In [ ]:
# 결측치가 포함
parsed.map(md => md.scores(0)).stats()

# 결측치 처리
import java.lang.Double.isNaN
parsed.map(md => md.scores(0)).filter(!isNaN(_)).stats()

# 9개의 score에 대해서 일괄처리함.
val stats = (0 until 9).map(i => {
  parsed.map(md => md.scores(i)).filter(!isNaN(_)).stats()
})

stats(1)
stats(8)

## Creating Reusable Code for Computing Summary Statistics

- Spark에서는 요약통계량을 계산하고, 유연하게 사용할 수 있는 유틸클래스를 제공함
- org.apache.spark.util.StatCounter
- 이 StatCounter 클래스는 결측치( NaN )값을 처리하지 못하므로 이것을 처리할 수 있는 클래스를 만들어보자.
- Serializable 을 상속받은 이유는 spark RDD 내부에서 인스턴스로 사용하기 위함.  
- RDD내부에 포함되는 인스턴스가 Serializable을 상속받지 않으면 에러가 발생함.

In [ ]:
import org.apache.spark.util.StatCounter
class NAStatCounter extends Serializable {
    val stats: StatCounter = new StatCounter()
    var missing: Long = 0
    
    def add(x: Double): NAStatCounter = {
        if (java.lang.Double.isNaN(x)) {
            missing += 1
        } else {
            stats.merge(x)
        }
        this
    }
    
    def merge(other: NAStatCounter): NAStatCounter = {
         stats.merge(other.stats)
         missing += other.missing
         this
    }
    override def toString = {
        "stats: " + stats.toString + " NaN: " + missing
    }
}

object NAStatCounter extends Serializable {
   def apply(x: Double) = new NAStatCounter().add(x)
}

In [ ]:
#  scala 파일로 만들어 놓은 class를 로드 
:load StatsWithMissing.scala

# object의 apply() 함수 호출
val nastats = NAStatCounter.apply(17.29)

# NAStatCounter의 생성자는 없지만, apply()함수가 있어서 Double타입의 변수에 적용됨.
val nastats = NAStatCounter(17.29)


val nas1 = NAStatCounter(10.0)
nas1.add(2.1)
val nas2 = NAStatCounter(Double.NaN)
nas1.merge(nas2)

In [ ]:
val arr = Array(1.0, Double.NaN, 17.29)
val nas = arr.map(d => NAStatCounter(d))

- parsed RDD내부에 있는 레코드에는 Array[Double] 있고, 이것을 Array[NAStatCounter] 형식으로 변환하자.

In [ ]:
val nasRDD = parsed.map(md => {
    md.scores.map(d => NAStatCounter(d))
})

- scala의 zip()함수를 이용해서 2개의 배열을 조합할 수 있음.
- zip() 함수의 기능을 예제로 확인함.

In [ ]:
val nas1 = Array(1.0,        Double.NaN).map(d => NAStatCounter(d))
val nas2 = Array(Double.NaN, 2.0       ).map(d => NAStatCounter(d))
nas1.zip(nas2)
val merged = nas1.zip(nas2).map(p => p._1.merge(p._2))

# 튜플에 대해서 case() 문을 사용해서 편리하게 사용할 수 있음.
val merged1 = nas1.zip(nas2).map { case (a, b) => a.merge(b) }

- scala의 reduce()함수는 T 타임의 2개의 인자를 받아서  2개의 인자를 가공해서 T타임의 값을 리턴해줌.
- collection객체 내부에 있는 모든 요소들에 대해서 반복적으로 수행함.
- 예) a = a + b 

In [ ]:
val nas = List(nas1, nas2)
val merged2 = nas.reduce((n1, n2) => {
    n1.zip(n2).map { case (a, b) => a.merge(b) }
})


val reduced = nasRDD.reduce((n1, n2) => {
    n1.zip(n2).map { case (a, b) => a.merge(b) }
})
reduced.foreach(println)

- StatsWithMissing.scala 에 추가해서  모듈화함.

In [ ]:
import org.apache.spark.rdd.RDD
def statsWithMissing(rdd: RDD[Array[Double]]): Array[NAStatCounter] = {
    val nastats = rdd.mapPartitions((iter: Iterator[Array[Double]]) => {
        val nas: Array[NAStatCounter] = iter.next().map(d => NAStatCounter(d))
        iter.foreach(arr => {
            nas.zip(arr).foreach { case (n, d) => n.add(d) }
        })
        Iterator(nas)
    })
    nastats.reduce((n1, n2) => {
        n1.zip(n2).map { case (a, b) => a.merge(b) }
    })
}


## Simple Variable Selection and Scoring

- statsWithMissing()함수를 이용해서  matches와 nonmatches 의 2가지 경우에 대해서 데이터의 분포가 어떻게 다른지 확인하자.

In [ ]:
val statsm = statsWithMissing(parsed.filter(_.matched).map(_.scores))
val statsn = statsWithMissing(parsed.filter(!_.matched).map(_.scores))

# 위의 코드와 동일한 기능을 함.
val matchedRDD = parsed.filter( a =>  a.matched == true ).map( b =>  b.scores)
val statsm = statsWithMissing(  matchedRDD  )

val notMatchedRDD = parsed.filter( a =>  a.matched != true ).map( b =>  b.scores)
val statsn = statsWithMissing( notMatchedRDD )

- matches와 nonmatches 일때의 데이터의 분포를 구별하기 위해서 평균의 차이를 구해보자.

In [ ]:
statsm.zip(statsn).map { case(m, n) =>
    (m.missing + n.missing, m.stats.mean - n.stats.mean)
}.foreach(println)

((1007, 0.2854...), 0)
((5645434,0.09104268062279874), 1)
((0,0.6838772482597568), 2)
((5746668,0.8064147192926266), 3)
((0,0.03240818525033484), 4)
((795,0.7754423117834044), 5)
((795,0.5109496938298719), 6)
((795,0.7762059675300523), 7)
((12843,0.9563812499852178), 8)

- 위의 통계량에서 데이터에서 2가지 중요한 특징을 찾을 수 있음
    - 중요한 차이가 어디에서 발생하고 있는지
    - 의미있는 값을 나타날수 있는 데이터가 무엇인지
- 1번인 경우에는  missing 데이터가 많고 차이가 상대적으로 적어서 유용해보이지 않음.
- 4번인 경우도 도움이 되지 않을 것 같음.
- 5번, 7번인 경우는 훌륭한 데이터로 보여짐.
- 2, 6, 8번은 유익해보임.

- 좋은 특성치인 2, 5, 6, 7, 8번에 대해서 값의 합을 구해보자.

In [ ]:
def naz(d: Double) = if (Double.NaN.equals(d)) 0.0 else d

case class Scored(md: MatchData, score: Double)

val ct = parsed.map(md => {
    val score = Array(2, 5, 6, 7, 8).map(i => naz(md.scores(i))).sum
    Scored(md, score)
})

ct.filter(s => s.score >= 4.0).map(s => s.md.matched).countByValue()
Map(false -> 637, true -> 20871)

ct.filter(s => s.score >= 2.0).map(s => s.md.matched).countByValue()
Map(false -> 596414, true -> 20931)

- score값을 조절해서 true match 된 자료를 필터링해서 분석에 사용할 수 있음.



- 다음장에서는 Spark MLlib을 다룸.